In [137]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import numpy

In [138]:
videoList = ["VID_20150123_172926_F.mp4.txt",
    "VID_20150123_173259_Q.mp4.txt",
    "VID_20150123_173329_F.mp4.txt",
    "VID_20150123_173358_Q.mp4.txt",
    "VID_20150123_173427_F.mp4.txt",
    "VID_20150123_173457_Q.mp4.txt",
    "VID_20150123_173556_Q.mp4.txt",
    "VID_20150123_173629_F.mp4.txt",
    "VID_20150123_173700_Q.mp4.txt",
    "VID_20150123_173757_Q.mp4.txt",
    "VID_20150123_173028_F.mp4.txt",
    "VID_20150123_172857_Q.mp4.txt",
    "VID_20150123_173727_F.mp4.txt",
    "VID_20150123_172827_F.mp4.txt",
    "VID_20150123_172956_Q.mp4.txt",
    "VID_20150123_173057_Q.mp4.txt",
    "VID_20150123_173127_F.mp4.txt",
    "VID_20150123_173157_Q.mp4.txt",
    "VID_20150123_173230_F.mp4.txt",
    "VID_20150123_173527_F.mp4.txt"]
fps = 30.323427803

In [139]:
def readFromFile(fileName):
    l =np.loadtxt(fileName)
    return l[0], l[1], l[2]

In [140]:
#will return a list of the indexes of the peaks
def peakList(colorList, frameThreshold):
    peaks = []
    for i in range(1,len(colorList)-1):
        #check to see if greater than immediate surroundings
        if colorList[i-1] <  colorList[i]  and colorList[i+1] <  colorList[i]:
            #if so, check every other frame in the threshold
            for j in range(2, frameThreshold+1):
                #check everything in threshold before - will break if not a peak
                if i-j>0:
                    if colorList[i]<colorList[i-j]:
                        break
                #check everything in threshold after
                if i+j<len(colorList):
                    if colorList[i]<colorList[i+j]:
                        break
            else:
                peaks.append(i)
    return peaks

In [141]:
def rrList(peakIndexes):
    rrlist = [];
    for i in range(1,len(peakIndexes)):
        rr = (peakIndexes[i]-peakIndexes[i-1])*1/fps
        rrlist.append(rr)
    return rrlist

In [142]:
sig1 = []
sig2 = []
signal1 = open("FullSignal1")
signal2 = open("FullSignal2")
filecontents = signal1.readlines()
for line in filecontents:
    num = line.strip('\n')
    sig1.append((int)(num))
    
filecontents = signal2.readlines()
for line in filecontents:
    num = line.strip('\n')
    sig2.append((int)(num))
    
for i in range(0,len(sig1)):
    sig1[i]= sig1[i]/1000.0
for i in range(0,len(sig2)):
    sig2[i]= sig2[i]/1000.0

In [143]:
def getDifference(segment,fullSignal, beginning):
    relDist = 0
    divisor=0
    for i in range(0,len(segment)):
        relDist += abs(segment[i]-fullSignal[i+beginning])
        divisor+=(fullSignal[i+beginning])
    relDist = relDist/divisor
        #seems to always return the same number when I include /(fullSignal[i+beginning]) FIXED
    return relDist
    

In [144]:
#we've decided to obtain rr intervals at a total frame threshold of 14 and using the color red
reds = []
for i in range(0,len(videoList)):
    r,g,b=readFromFile(videoList[i])
    reds.append(r)
frameThresh = 7

rrLists = []
for i in range(0,len(reds)):
    rrLists.append(rrList(peakList(reds[i],7)))

In [145]:
def findBestMatch(segment,fullSignal):
    diffs = []
    for e in range(0,len(fullSignal)-len(segment)+1):
        diffs.append(getDifference(segment,fullSignal,e))
    return diffs.index(min(diffs)),min(diffs)

In [148]:
#Show the similarity of the best match
print findBestMatch(rrLists[1],sig1)
print findBestMatch(rrLists[1],sig2)
print rrLists[1]
print sig2[289:289+len(rrLists[1])]

(772, 0.28079338710067797)
(289, 0.013529011419822622)
[0.9233784578018539, 0.9233784578018539, 0.9233784578018539, 0.8904006557375019, 0.8904006557375019, 0.9233784578018539, 0.8244450516087981, 0.7584894474800943, 0.7584894474800943, 0.8244450516087981, 0.9563562598662059, 0.9563562598662059, 0.9563562598662059, 1.0223118639949096, 1.0223118639949096, 0.9893340619305577, 0.9893340619305577, 1.0223118639949096, 1.0223118639949096, 0.9563562598662059, 0.9563562598662059, 0.9893340619305577, 0.9893340619305577, 0.9563562598662059, 0.9563562598662059, 0.9563562598662059, 0.9893340619305577, 0.9563562598662059, 0.8904006557375019, 0.9563562598662059]
[0.911, 0.922, 0.925, 0.893, 0.907, 0.896, 0.842, 0.789, 0.775, 0.833, 0.976, 0.943, 0.976, 1.033, 1.034, 0.994, 1.0, 1.032, 1.022, 0.969, 0.978, 1.0, 1.004, 0.96, 0.965, 0.976, 0.992, 0.947, 0.933, 0.954]


In [161]:
#Videos shown to have the least error:
bestVideos = [1, 3, 5, 6, 8, 9, 10, 14, 17]

bestMatches = []
#contains lists that are defined by [segment,fullSignal,index,diffValue]
for video in bestVideos:
    match1,diffValue1 = findBestMatch(rrLists[video],sig1)
    match2,diffValue2 = findBestMatch(rrLists[video],sig2)
    
    if diffValue1<diffValue2:
        bestMatches.append([video,"Signal 1",match1,diffValue1])
    else:
        bestMatches.append([video,"Signal 2",match2,diffValue2])
for match in bestMatches:
    print match

[1, 'Signal 2', 289, 0.013529011419822622]
[3, 'Signal 2', 351, 0.019377144377706355]
[5, 'Signal 2', 413, 0.01883560690755128]
[6, 'Signal 2', 474, 0.010777444148100869]
[8, 'Signal 2', 541, 0.018561124446814122]
[9, 'Signal 2', 601, 0.021092907103444337]
[10, 'Signal 1', 234, 0.026683958330108715]
[14, 'Signal 2', 96, 0.02792884157508321]
[17, 'Signal 2', 223, 0.019304002573009633]


In [173]:
#trying with videos of slightly lesser quality
goodVideos = [0, 4, 11, 13, 15, 16,18]
goodMatches = []
#contains lists that are defined by [segment,fullSignal,index,diffValue]
for video in goodVideos:
    match1,diffValue1 = findBestMatch(rrLists[video],sig1)
    match2,diffValue2 = findBestMatch(rrLists[video],sig2)
    
    if diffValue1<diffValue2:
        goodMatches.append([video,"Signal 1",match1,diffValue1])
    else:
        goodMatches.append([video,"Signal 2",match2,diffValue2])
for match in goodMatches:
    print match
#uh oh! Two of them seem to have the exact same placement?
for i in range(0,len(rrLists[4])):
    print rrLists[4][i],rrLists[11][i]
print "----------------------------"
r4,g,b = readFromFile(videoList[4])
r11,g,b = readFromFile(videoList[11])

for i in range(0,len(rrList(peakList(r4,7)))):
    print rrList(peakList(r4,7))[i],rrList(peakList(r11,7))[i]
#seem to be nearly identical w/ a few switched... thats a problem
print videoList[4]
print videoList[11]

[0, 'Signal 1', 145, 0.0516058667492009]
[4, 'Signal 2', 32, 0.04014260245695516]
[11, 'Signal 2', 32, 0.04568676033430361]
[13, 'Signal 1', 55, 0.05740598670740003]
[15, 'Signal 2', 159, 0.031288477618948055]
[16, 'Signal 1', 325, 0.04730528985916612]
[18, 'Signal 1', 416, 0.041679633643246734]
0.626578239223 0.626578239223
0.560622635094 0.560622635094
0.494667030965 0.494667030965
0.923378457802 0.923378457802
0.923378457802 0.923378457802
0.956356259866 0.956356259866
0.989334061931 0.989334061931
0.923378457802 0.923378457802
0.791467249544 0.791467249544
0.857422853673 0.824445051609
0.857422853673 0.857422853673
0.857422853673 0.857422853673
0.857422853673 0.824445051609
0.923378457802 0.923378457802
1.02231186399 1.02231186399
0.989334061931 0.956356259866
0.956356259866 0.956356259866
0.989334061931 0.989334061931
0.989334061931 0.956356259866
0.989334061931 0.989334061931
0.923378457802 0.923378457802
0.857422853673 0.857422853673
0.890400655738 0.857422853673
0.923378457802 

In [178]:
#getting the time for the placement of the video
seconds = 0
for i in range(0,351):
    seconds += sig2[i]
seconds

326.26700000000005